ToDo:

Plot important dates against our output: https://www.policyuncertainty.com/media/US_Annotated_Series.pdf

1) Bush Election
2) 9/11
3) Gulf War II
4) Stimulus Debate ~2007/ 2008
5) Lehman/ TARP ~2008/ 2009
6) Eurozone Crisis/ US Midterm Elections ~Nov 2010
7) Debt Ceiling Debate ~2011/2012
8) Fiscal Cliff ~2013
9) Govt. Shutdown ~2013/ 2014
10) Brexit ~2016
11) Trump Election ~Nov 2016
12) US out of TPP ~2016/ 2017
13) Tarrifs 2018

Market Dates: https://www.timetoast.com/timelines/important-dates-in-the-history-of-the-u-s-stock-markets

1) Dot Com Bubble = Mar 10, 2000
2) NASDAQ Drop 356 Points = Apr 14, 2000
3) Hurricane Katrina = Aug 25, 2005
4) 2008 Market Selloff = 09/29/2008, 10/08/2008, 10/15/2008, 10/22/2008, 12/01/2008, 12/10/2008
5) 2011 Market Selloff = 08/04/2011, 08/08/2011, 09/17/2011, 12/31/2011
6) Derive other dates from SPX data

https://fraser.stlouisfed.org/timeline/covid-19-pandemic

https://fraser.stlouisfed.org/timeline/financial-crisis

In [6]:
%config Completer.use_jedi = False #for intellisense compatibility w/ Jupyter Notebook
%matplotlib inline

from sklearn.preprocessing import StandardScaler
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipdb
import copy
import datetime as dt
from iGetByWithALittle import find_target_columns, find_new_obs_percent_chg, find_new_vintage_percent_chg, find_index_in_Excel_File_dict

Read in FOMC Meeting Dates

In [2]:
df = pd.read_csv('fomc_dates.csv')
FOMC_meets = df['fomc_date'].tolist()
FOMC_dates = [dt.datetime.strptime(meet,'%m/%d/%Y') for meet in FOMC_meets]

Read in and Format Vintage GDP Data from ALFRED File

In [91]:
# URL: https://alfred.stlouisfed.org/series/downloaddata?seid=GDPC1
# Note: Must select 'All' in the Vintage Dates section. Data as of 1/28/2021

# read in FRED data
df = pd.read_csv('GDPC1_2_Vintages_Starting_1991_12_04.txt', sep='\t', na_values='.')

# set index to observation date
df.set_index('observation_date', inplace=True)

# discard data prior to 1999 as our FOMC data begin in 2000
df = df.loc['1999-01-01':]

# drop any remaining columns with no observations
df = df.dropna(how='all', axis=1)

# calculate vintage percent changes with helper function
vintage_gdp_changes = find_new_vintage_percent_chg(df, FOMC_dates, column_A_name='gdp-1', column_B_name='gdp-0')
vintage_gdp_changes.to_csv('vintage_gdp_changes.csv')
vintage_gdp_changes

,gdp-1,gdp-0
2000-02-02,1.389743,1.419005
2000-03-21,1.389743,1.688650
2000-05-16,1.772562,1.321206
2000-06-28,1.772562,1.322312
2000-08-22,1.338910,1.272874
...,...,...
2020-03-15,0.521779,0.519729
2020-04-29,0.521779,0.527469
2020-06-10,0.527469,-1.286382
2020-07-29,0.527469,-1.272539


Read in and Format Vintage GDP Chain-Type Price Index Data from ALFRED File

In [50]:
#URL: https://alfred.stlouisfed.org/series/downloaddata?seid=GDPCTPI
#NOTE: Must select 'All' in the Vintage Dates section. Data as of 1/28/2021

# read in FRED data
df = pd.read_csv('GDPCTPI_2_Vintages_Starting_1996_01_19.txt', sep='\t', na_values='.')

# set index to observation date
df.set_index('observation_date', inplace=True)

#discard data prior to 1999 as our FOMC data begins in 2000
df = df.loc['1999-01-01':]

# drop any remaining columns with no observations
df = df.dropna(how='all', axis=1)

vintage_gdp_price_changes = find_new_vintage_percent_chg(df, FOMC_dates, column_A_name='price-1', column_B_name='price-0')
vintage_gdp_price_changes.to_csv('vintage_gdp_price_changes.csv')

Read in and Format US Economic Policy Uncertainty Index Data from ALFRED

In [19]:
#URL: https://alfred.stlouisfed.org/series/downloaddata?seid=USEPUINDXD
#Note: Cannot select all here as too many vintages exists. Data as of 1/28/2021.
# These data don't strongly revise so we calculate on the latest vintage

# read in FRED data
df = pd.read_csv('USEPUINDXD_2_Vintages_Starting_2018_06_29.txt', sep='\t', na_values='.')

# set index to observation date
df.set_index('observation_date', inplace=True)
df.index = pd.to_datetime(df.index)
df = df.iloc[:, -1:]
df = df.rename(columns={'USEPUINDXD_20210128': 'EPU'})
df['EPU 30 Day MA'] = df.rolling(window=30)['EPU'].mean()
df['EPU Std 30 Day MA'] = StandardScaler().fit_transform(df[['EPU 30 Day MA']])

epu_df = df[['EPU Std 30 Day MA']]
# epu_df.to_csv('epu_df.csv')

FOMC_df = pd.DataFrame(index=FOMC_meets)
FOMC_df.index = pd.to_datetime(FOMC_df.index)
FOMC_df.index.name = 'observation_date'

FOMC_epu = pd.merge_asof(FOMC_df, epu_df, on = 'observation_date')
FOMC_epu.set_index('observation_date', inplace=True)
# FOMC_epu

Read in and Format S&P 500 Data from Yahoo

In [20]:
df = pd.read_csv('^GSPC.csv')
df.set_index('Date', inplace=True)
df.index = pd.to_datetime(df.index)
df = df.rename(columns={'Close': 'SPX'})
df['SPX 30 Day MA'] = df.rolling(window=30)['SPX'].mean()
df['SPX Std 30 Day MA'] = StandardScaler().fit_transform(df[['SPX 30 Day MA']])


df.index.name = 'observation_date'
spx_df = df[['SPX Std 30 Day MA']]

FOMC_spx = pd.merge_asof(FOMC_df, spx_df, on = 'observation_date')
FOMC_spx.set_index('observation_date', inplace=True)
# FOMC_spx

Read in and Format Mean GDP Forecast Data from FRB Philadelphia

In [142]:
#URL: https://www.philadelphiafed.org/surveys-and-data/rgdp
#Note: 'Mean Responses' as of 1/10/2021
df = pd.read_excel(io='Mean_RGDP_Level.xlsx', sheet_name='Mean_Level') #read in data
philly_dates = pd.read_csv('philly_release_dates.csv')
df = df[df['YEAR'].notna()] #drop last col which shows up as NaT in index (not sure why this shows up...)
df['YEAR'] = df['YEAR'].astype(int).astype(str) #cast year to string for Period index
df['QUARTER'] = df['QUARTER'].astype(int).astype(str) #cast quarter to string for Period index
df['Period'] = df[['YEAR', 'QUARTER']].agg('-Q'.join, axis=1) #combine year and quarter data into Period
df['Period'] = pd.to_datetime(df['Period']) #set Period to datetime
df.set_index('Period', inplace=True) #set Period as df index
df = df['1999-10-01':]
philly_dates = philly_dates.set_index(df.index)
df['observation_dates'] = philly_dates['Period']
df = df.set_index(df['observation_dates'])
df.index = pd.to_datetime(df.index)
GDP_cols_list = ['RGDP1', 'RGDP2', 'RGDP3'] #identify columns we want to retain and lag
df = df.filter(GDP_cols_list, axis=1) #retain columns we want
df[GDP_cols_list] = df[GDP_cols_list].shift(-1) #lag values to correct index (as Philly Fed index is to the forecast, not GDP period
df['gdp+1_mean'] = df[['RGDP1', 'RGDP2']].apply(lambda row: (row.iloc[1]-row.iloc[0])/row.iloc[0]*100, axis=1) #calculate pct change from actual to forecast 1evels 
df['gdp+2_mean'] = df[['RGDP2', 'RGDP3']].apply(lambda row: (row.iloc[1]-row.iloc[0])/row.iloc[0]*100, axis=1) #calculate pct change from actual to forecast levels 
mean_GDP_forecasts = ['gdp+1_mean', 'gdp+2_mean'] #identify columns to retain
mean_GDP_forecasts = df.filter(mean_GDP_forecasts, axis=1) #filter for wanted columns
mean_GDP_forecasts.index.name = 'observation_date'
mean_GDP_forecasts.to_csv('mean_GDP_forecasts.csv')

FOMC_df = pd.DataFrame(index=FOMC_meets)
FOMC_df.index = pd.to_datetime(FOMC_df.index)
FOMC_df.index.name = 'observation_date'

FOMC_gdp = pd.merge_asof(FOMC_df, mean_GDP_forecasts, on = 'observation_date',direction='backward')
FOMC_gdp.set_index('observation_date', inplace=True)
# FOMC_gdp['2000-01-01':].head(15)

C:\Users\micha\miniconda3\envs\FOMC\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [148]:
macro_df = vintage_gdp_changes.join(FOMC_gdp)
macro_df = macro_df.join(vintage_gdp_price_changes)
macro_df = macro_df.join(FOMC_epu)
macro_df = macro_df.join(FOMC_spx)
macro_df

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!Double check this on Monday!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

,gdp-1,gdp-0,gdp+1_mean,gdp+2_mean,price-1,price-0,EPU Std 30 Day MA,SPX Std 30 Day MA
2000-02-02,1.389743,1.419005,0.741383,0.790307,0.268895,0.507614,-0.900118,1.279875
2000-03-21,1.389743,1.688650,1.031194,0.901730,0.268895,0.498037,-0.883935,1.225654
2000-05-16,1.772562,1.321206,1.031194,0.901730,0.487339,0.665652,-0.918470,1.299294
2000-06-28,1.772562,1.322312,0.802572,0.838087,0.487339,0.665652,-0.736161,1.296398
2000-08-22,1.338910,1.272874,0.802572,0.838087,0.751236,0.621645,-0.963599,1.338053
...,...,...,...,...,...,...,...,...
2020-03-15,0.521779,0.519729,-9.110431,2.373880,0.451089,0.314167,1.114888,3.606619
2020-04-29,0.521779,0.527469,-9.110431,2.373880,0.451089,0.316829,7.910614,2.947602
2020-06-10,0.527469,-1.286382,4.508601,1.348393,0.316829,0.341484,5.015778,3.387269
2020-07-29,0.527469,-1.272539,4.508601,1.348393,0.316829,0.348562,4.407042,3.635930


Read in and Format Median GDP Forecast Data from FRB Philadelphia

In [ ]:
#URL: https://www.philadelphiafed.org/surveys-and-data/rgdp
#Note: 'Median Responses' as of 1/10/2021
df = pd.read_excel(io='Median_RGDP_Level.xlsx', sheet_name='Median_Level') #read in data
df = df[df['YEAR'].notna()] #drop last col which shows up as NaT in index (not sure why this shows up...)
df['YEAR'] = df['YEAR'].astype(int).astype(str) #cast year to string for Period index
df['QUARTER'] = df['QUARTER'].astype(int).astype(str) #cast quarter to string for Period index
df['Period'] = df[['YEAR', 'QUARTER']].agg('-Q'.join, axis=1) #combine year and quarter data into Period
df['Period'] = pd.to_datetime(df['Period']) #set Period to datetime
df.set_index('Period', inplace=True) #set Period as df index
df.index = df.index.to_period("Q") #format index
GDP_cols_list = ['RGDP1', 'RGDP2', 'RGDP3'] #identify columns we want to retain and lag
df = df.filter(GDP_cols_list, axis=1) #retain columns we want
df[GDP_cols_list] = df[GDP_cols_list].shift(-1) #lag values to correct index (as Philly Fed index is to the forecast, not GDP period) 
df = df[df['RGDP1'].notna()] #remove rows where RGDP1 has NaN
df['1Q Ahead Median GDP Forecast'] = df[['RGDP1', 'RGDP2']].apply(lambda row: (row.iloc[1]-row.iloc[0])/row.iloc[0]*100, axis=1) #calculate pct change from actual to forecast 1
df['2Q Ahead Median GDP Forecast'] = df[['RGDP2', 'RGDP3']].apply(lambda row: (row.iloc[1]-row.iloc[0])/row.iloc[0]*100, axis=1) #calculate pct change from actual to forecast 1
median_GDP_forecasts = ['1Q Ahead Median GDP Forecast', '2Q Ahead Median GDP Forecast'] #identify columns to retain
median_GDP_forecasts = df.filter(median_GDP_forecasts, axis=1) #filter for wanted columns
# median_GDP_forecasts

Read in and Format Mean GDP Price Index Forecast Data from FRB Philadelphia

In [ ]:
#URL: https://www.philadelphiafed.org/surveys-and-data/pgdp
#Note: 'Mean Responses' as of 1/12/2021
df = pd.read_excel(io='Mean_PGDP_Level.xlsx', sheet_name='Mean_Level') #read in data
df = df[df['YEAR'].notna()] #drop last col which shows up as NaT in index (not sure why this shows up...)
df['YEAR'] = df['YEAR'].astype(int).astype(str) #cast year to string for Period index
df['QUARTER'] = df['QUARTER'].astype(int).astype(str) #cast quarter to string for Period index
df['Period'] = df[['YEAR', 'QUARTER']].agg('-Q'.join, axis=1) #combine year and quarter data into Period
df['Period'] = pd.to_datetime(df['Period']) #set Period to datetime
df.set_index('Period', inplace=True) #set Period as df index
df.index = df.index.to_period("Q") #format index
PGDP_cols_list = ['PGDP1', 'PGDP2', 'PGDP3'] #identify columns we want to retain and lag
df = df.filter(PGDP_cols_list, axis=1) #retain columns we want
df[PGDP_cols_list] = df[PGDP_cols_list].shift(-1) #lag values to correct index (as Philly Fed index is to the forecast, not GDP period
df = df[df['PGDP1'].notna()] #remove rows where RGDP1 has NaN
df['1Q Ahead Mean GDP Price Forecast'] = df[['PGDP1', 'PGDP2']].apply(lambda row: (row.iloc[1]-row.iloc[0])/row.iloc[0]*100, axis=1) #calculate pct change from actual to forecast 1evels 
df['2Q Ahead Mean GDP Price Forecast'] = df[['PGDP2', 'PGDP3']].apply(lambda row: (row.iloc[1]-row.iloc[0])/row.iloc[0]*100, axis=1) #calculate pct change from actual to forecast levels 
mean_PGDP_forecasts = ['1Q Ahead Mean GDP Price Forecast', '2Q Ahead Mean GDP Price Forecast'] #identify columns to retain
mean_PGDP_forecasts = df.filter(mean_PGDP_forecasts, axis=1) #filter for wanted columns
# mean_PGDP_forecasts

Read in and Format Median GDP Price Index Forecast Data from FRB Philadelphia

In [ ]:
#URL: https://www.philadelphiafed.org/surveys-and-data/pgdp
#Note: 'Median Responses' as of 1/12/2021
df = pd.read_excel(io='Median_PGDP_Level.xlsx', sheet_name='Median_Level') #read in data
#print(df)
df = df[df['YEAR'].notna()] #drop last col which shows up as NaT in index (not sure why this shows up...)
df['YEAR'] = df['YEAR'].astype(int).astype(str) #cast year to string for Period index
df['QUARTER'] = df['QUARTER'].astype(int).astype(str) #cast quarter to string for Period index
df['Period'] = df[['YEAR', 'QUARTER']].agg('-Q'.join, axis=1) #combine year and quarter data into Period
df['Period'] = pd.to_datetime(df['Period']) #set Period to datetime
df.set_index('Period', inplace=True) #set Period as df index
df.index = df.index.to_period("Q") #format index
PGDP_cols_list = ['PGDP1', 'PGDP2', 'PGDP3'] #identify columns we want to retain and lag
df = df.filter(PGDP_cols_list, axis=1) #retain columns we want
df[PGDP_cols_list] = df[PGDP_cols_list].shift(-1) #lag values to correct index (as Philly Fed index is to the forecast, not GDP period
df = df[df['PGDP1'].notna()] #remove rows where RGDP1 has NaN
df['1Q Ahead Median GDP Price Forecast'] = df[['PGDP1', 'PGDP2']].apply(lambda row: (row.iloc[1]-row.iloc[0])/row.iloc[0]*100, axis=1) #calculate pct change from actual to forecast 1evels 
df['2Q Ahead Median GDP Price Forecast'] = df[['PGDP2', 'PGDP3']].apply(lambda row: (row.iloc[1]-row.iloc[0])/row.iloc[0]*100, axis=1) #calculate pct change from actual to forecast levels 
median_PGDP_forecasts = ['1Q Ahead Median GDP Price Forecast', '2Q Ahead Median GDP Price Forecast'] #identify columns to retain
median_PGDP_forecasts = df.filter(median_PGDP_forecasts, axis=1) #filter for wanted columns
# median_PGDP_forecasts

Combine GDP Data from ALFRED & Philly Fed

In [ ]:
GDP_data = GDP_actuals.join(mean_GDP_forecasts) #left join the GDP_acuals and mean_GDP_forecasts
GDP_data = GDP_data.join(median_GDP_forecasts) #left join the GDP_acuals and median_GDP_forecasts
GDP_data = GDP_data.loc['2000Q1':] #remove rows outside of our period
GDP_data.index = GDP_data.index.strftime('%m/%d/%Y') #set index format to m/d/yyyy

Combine PCE Data from ALFRED & Philly Fed

In [ ]:
PCE_forecasts = mean_PCE_forecasts.join(median_PCE_forecasts)
PCE_data = PCE_actuals.join(PCE_forecasts)
# # Test for data continuity
# print('Corr Median & FRED Actual', PCE_data['Curr PCE Pct Chg'].corr(PCE_data['COREPCE - Median'])) #check for data continuity across the FRED actual and Philly Fed forcast data
# print('Corr Median & FRED Actual', PCE_data['Curr PCE Pct Chg'].corr(PCE_data['COREPCE - Mean'])) #check for data continuity across the FRED actual and Philly Fed forcast data
PCE_data_to_drop = ['COREPCE - Median','COREPCE - Mean']
PCE_data = PCE_data.drop(PCE_data_to_drop, axis=1)
PCE_data.index = PCE_data.index.strftime('%m/%d/%Y') #set index format to m/d/yyyy

Combine PGDP Data from ALFRED & Philly Fed

In [ ]:
PGDP_forecasts = mean_PGDP_forecasts.join(median_PGDP_forecasts)
PGDP_data = PGDP_actuals.join(PGDP_forecasts)
PGDP_data = PGDP_data.loc['2000Q1':] #remove rows outside of our period
PGDP_data.index = PGDP_data.index.strftime('%m/%d/%Y') #set index format to m/d/yyyy

Combine Datasets for Shock Regression

In [ ]:
'''
    Miles, this one is for you. Call the ball. :)
'''

macro_data = GDP_data.join(PGDP_data)
# macro_data
# macro_data.to_csv('macro_data.csv')

In [ ]:
macro_data